In [1]:
import keras
import numpy as np
import os
import pandas as pd
import pickle
import sys
import tensorflow as tf

from keras import layers
from keras import models
from keras import utils
from keras.utils import to_categorical
from tensorflow import keras

# This tokenizer is nice, but could cause problems.
try:
    from nltk.tokenize.moses import MosesDetokenizer
    detokenizer = MosesDetokenizer()
    use_moses_detokenizer = True
except:
    use_moses_detokenizer = False

# Parameters

In [2]:
# folders
home = os.getenv("HOME")
nlp_repo = os.path.join(home, 'git/nlp-product-sentiment-classification')

# Data
train_csv_path = os.path.join(nlp_repo, 'data/03_processed/Train.csv')
train_descr = pd.read_csv(train_csv_path)

# Encoded Tokens
preprocessed_corpus_path = os.path.join(
    nlp_repo, 'data/03_processed/product_descr_preprocessed.p')

# Training parameters
train_anyway = False
model_path = "product_descr.h5"
#dataset_size = 5000
sequence_length = 30
epochs = 10
batch_size = 128
hidden_size = 1000

# Turning the Encoded Sequences into a Tensorflow Dataset

In [3]:
indices, vocabulary = pd.read_pickle(preprocessed_corpus_path)

Sources:
* TF dataset: https://stackoverflow.com/questions/58362316/how-do-i-go-from-pandas-dataframe-to-tensorflow-batchdataset-for-nlp

In [4]:
train_seqs = tf.keras.preprocessing.sequence.pad_sequences(
    indices, padding='post', value=len(vocabulary))
train_labels = train_descr['Sentiment'].to_numpy().flatten()

In [5]:
print(train_seqs)
print(train_labels)

[[ 154 3719  294 ... 7119 7119 7119]
 [   4    1   24 ... 7119 7119 7119]
 [ 451    6   22 ... 7119 7119 7119]
 ...
 [   4    1    4 ... 7119 7119 7119]
 [ 204  256   16 ... 7119 7119 7119]
 [   5   13 7118 ... 7119 7119 7119]]
[2 2 2 ... 2 2 2]


In [6]:
train_ds = tf.data.Dataset.from_tensor_slices((train_seqs, train_labels))

#### Check, if transformation went ok

In [7]:
test_descr, test_label = list(train_ds.take(1))[0]

In [8]:
test_descr.numpy()

array([ 154, 3719,  294,  450,   17,   11,   18,    1,    2,    0, 7119,
       7119, 7119, 7119, 7119, 7119, 7119, 7119, 7119, 7119, 7119, 7119,
       7119, 7119, 7119], dtype=int32)

In [9]:
test_label.numpy()

2

In [10]:
# ToDo: import from utils

In [11]:
def decode_indices(indices, vocabulary):
    """ Decodes a sequence of indices and returns a string. """
    # adding an extra item for padding in order to keep the decoding correct after padding
    # if we use the default value 0 for padding, it would be translated to one of our tokens
    vocabulary.append('padding')
    decoded_tokens = [vocabulary[index] for index in indices]
    if use_moses_detokenizer == True:
        return detokenizer.detokenize(decoded_tokens, return_str=True)
    else:
        return " ".join(decoded_tokens)

In [12]:
decoded_test_descr = decode_indices(test_descr, vocabulary)
decoded_test_descr

'web designeruas guide ios android app today mention link sxsw padding padding padding padding padding padding padding padding padding padding padding padding padding padding padding'

# Bag of Words